In [ ]:
import os
import shutil

# Paths to the original folder and the new folder
original_folder = "/home/gs285/AIPI_HW/annotations_utensil"
new_folder = "/home/gs285/AIPI_HW/modified_annotations_utensil"

# Create the new folder if it doesn't exist
os.makedirs(new_folder, exist_ok=True)

# Loop through all .txt files in the original folder
for filename in os.listdir(original_folder):
    if filename.endswith(".txt"):  # Process only .txt files
        original_file_path = os.path.join(original_folder, filename)
        new_file_path = os.path.join(new_folder, filename)

        # Read the content of the original file
        with open(original_file_path, "r") as file:
            lines = file.readlines()

        # Modify the class ID (first element) in each line to 3
        updated_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts:  # Ensure the line is not empty
                parts[0] = "3"  # Change the class ID to 3
                updated_lines.append(" ".join(parts) + "\n")

        # Write the modified content to the new file
        with open(new_file_path, "w") as new_file:
            new_file.writelines(updated_lines)

print(f"All annotation files have been modified and saved to: {new_folder}")


**Prepare the dataset**

In [1]:
import os
import random
import shutil

# Define paths
images_folder = "yolo_data/images"
labels_folder = "yolo_data/labels"
train_images_folder = "yolo_data_split/images/train"
val_images_folder = "yolo_data_split/images/val"
train_labels_folder = "yolo_data_split/labels/train"
val_labels_folder = "yolo_data_split/labels/val"

# Create directories for train and val splits
os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(val_images_folder, exist_ok=True)
os.makedirs(train_labels_folder, exist_ok=True)
os.makedirs(val_labels_folder, exist_ok=True)

# Get list of image files and corresponding labels
image_files = os.listdir(images_folder)
image_files = [f for f in image_files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'))]  # Filter out only the image files

# Shuffle the dataset
random.seed(42)  # For reproducibility
random.shuffle(image_files)

# Define the split ratio
train_ratio = 0.8  # 80% for training, 20% for validation
split_index = int(len(image_files) * train_ratio)

# Split the data
train_images = image_files[:split_index]
val_images = image_files[split_index:]

# Move images and labels to corresponding folders
def move_files(image_list, destination_image_folder, destination_label_folder):
    for image_file in image_list:
        # Move the image file
        image_path = os.path.join(images_folder, image_file)
        shutil.copy(image_path, os.path.join(destination_image_folder, image_file))
        
        # Move the corresponding label file
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(labels_folder, label_file)
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(destination_label_folder, label_file))
        else:
            print(f"Warning: Label for {image_file} not found. Skipping.")

# Move training data
move_files(train_images, train_images_folder, train_labels_folder)

# Move validation data
move_files(val_images, val_images_folder, val_labels_folder)

print(f"Training set: {len(train_images)} images")
print(f"Validation set: {len(val_images)} images")


Training set: 240 images
Validation set: 60 images


In [2]:
import matplotlib.pyplot as plt

# Function to visualize predictions on the validation set
def visualize_predictions(model, val_images_folder, epoch):
    print(f"\nVisualizing predictions at epoch {epoch}...")
    # Select a few validation images to visualize predictions
    for i, image_file in enumerate(os.listdir(val_images_folder)[:8]):  # Use first 8 images
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')):
            print(f"Skipping unsupported file: {image_file}")
            continue  # Skip non-image files
        
        image_path = os.path.join(val_images_folder, image_file)
        results = model(image_path)  # Generate predictions
        result_img = results[0].plot()  # This renders an annotated NumPy image
        
        # Display predictions using matplotlib
        plt.figure(figsize=(10, 10))
        plt.imshow(result_img)
        plt.axis('off')
        plt.title(f"Predictions at Epoch {epoch}")
        plt.show()

In [11]:
from ultralytics import YOLO

# Load the pre-trained YOLO model
model = YOLO("yolov8n.pt")

# Path to validation images
val_images_folder = "yolo_data_split/images/val"

# Set total number of epochs and batch size
total_epochs = 1
epoch_step = 1  # Number of epochs per incremental training step

for epoch in range(0, total_epochs, epoch_step):
    print(f"\nTraining from epoch {epoch + 1} to {epoch + epoch_step}...")
    # Train the model for epoch_step epochs
    results = model.train(
        data="/home/gs285/AIPI_HW/custom_dataset.yaml",  # Path to the custom dataset YAML file
        epochs=epoch_step,  # Train for this many epochs in each step
        batch=8,  # Batch size
        imgsz=640,  # Image size for training
        name="yolov8_custom"  # Name of the experiment run
    )
    # # Visualize predictions on the validation set every 5 epochs
    # visualize_predictions(model, val_images_folder, epoch + epoch_step)

# Evaluate the final model on the validation set
val_results = model.val()

# # Extract performance metrics from the results_dict() method
results_dict = val_results.results_dict

# Get mAP, speed, and model size
mAP_50_95 = results_dict.get("metrics/mAP50-95(B)", None)  # mAP across IoUs
mAP_50  = results_dict.get("metrics/mAP50(B)", None)  # Use .get() to avoid KeyErrors

# Extract speed (inference time per image in ms)
speed = val_results.speed.get("inference", None)  # Speed in ms/img

# Print the evaluation metrics
print(f"Validation mAP@0.5:0.95: {mAP_50_95}")
print(f"Validation mAP@0.5: {mAP_50}")
print(f"Training speed (ms/img): {speed}")


# Save the model temporarily to get its size
temp_model_path = "temp_model.pt"
model.save(temp_model_path)  # Save the model weights
# Get the size in MB
model_size = os.path.getsize(temp_model_path) / (1024 * 1024)  # Convert bytes to MB
# Clean up the temporary file
os.remove(temp_model_path)
print(f"Model size: {model_size:.2f} MB")

# Export the trained model to ONNX format
model.export(format="onnx")


Training from epoch 1 to 1...
New https://pypi.org/project/ultralytics/8.3.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.1 🚀 Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/gs285/AIPI_HW/custom_dataset.yaml, epochs=1, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8_custom30, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None,

train: Scanning /home/gs285/AIPI_HW/yolo_data_split/labels/train.cache... 240 images, 0 backgrounds, 1 corrupt: 100%|██████████| 240/240 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/gs285/AIPI_HW/yolo_data_split/images/train/notebook-xps-13-9340-t-sl-gallery-2 (1).png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     3.4629      5.4863       6.403      8.2337      3.4307      7.8165      5.1152      1.5922]



val: Scanning /home/gs285/AIPI_HW/yolo_data_split/labels/val.cache... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]


Plotting labels to runs/detect/yolov8_custom30/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 63 weight(decay=0.0), 70 weight(decay=0.0005), 69 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 6 dataloader workers
Logging results to runs/detect/yolov8_custom30
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.80it/s]


                   all         60        144     0.0401      0.521       0.11     0.0522
                laptop         51        118     0.0769      0.119      0.111     0.0519
                 mouse         23         26    0.00326      0.923      0.108     0.0525
Speed: 0.2ms preprocess, 4.9ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to runs/detect/yolov8_custom30
Ultralytics 8.3.1 🚀 Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp, 12190MiB)
Model summary (fused): 186 layers, 2,685,148 parameters, 0 gradients, 6.8 GFLOPs


val: Scanning /home/gs285/AIPI_HW/yolo_data_split/labels/val.cache... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:00<00:00, 11.35it/s]


                   all         60        144     0.0318      0.491      0.103     0.0513
                laptop         51        118     0.0604      0.136      0.111     0.0558
                 mouse         23         26    0.00318      0.846     0.0959     0.0469
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/yolov8_custom302
Validation mAP@0.5:0.95: 0.05133518285984482
Validation mAP@0.5: 0.10347281984538868
Training speed (ms/img): 3.033188978830973
Model size: 5.26 MB
Ultralytics 8.3.1 🚀 Python-3.8.20 torch-2.4.1+cu121 CPU (Intel Xeon E5-2687W v4 3.00GHz)

PyTorch: starting from 'runs/detect/yolov8_custom30/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (5.4 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.34...
ONNX: export success ✅ 1.1s, saved as 'runs/detect/yolov8_custom30/weights/best.onnx' (10.5 MB)

Export complete (1.2s)
Result

'runs/detect/yolov8_custom30/weights/best.onnx'